## Text Classification Inference using Batch Endpoints

This sample shows how to run inference in batch model for `text-classification` task.

### Task
`text-classification` is generic task type that can be used for scenarios such as sentiment analysis, emotion detection, grammar checking, spam filtering, etc. In this example, we will test for entailment v/s contradiction, meaning given a premise sentence and a hypothesis sentence, the task is to predict whether the premise entails the hypothesis (entailment), contradicts the hypothesis (contradiction), or neither (neutral). 

### Inference data
The Multi-Genre Natural Language Inference Corpus, or MNLI is a crowd sourced collection of sentence pairs with textual entailment annotations.The [MNLI](https://huggingface.co/datasets/glue) dataset is a subset of the larger [General Language Understanding Evaluation](https://gluebenchmark.com/) dataset. A copy of this dataset is available in the [glue-mnli](./glue-mnli/) folder.

### Model
Look for models tagged with `text-classification` in the system registry. Just looking for `text-classification` is not sufficient, you need to check if the model is specifically finetuned for  entailment v/s contradiction by studying the model card and looking at the input/output samples or signatures of the model. In this notebook, we use the `microsoft-deberta-base-mnli` model.

  
### Outline
* Setup pre-requisites.
* Pick a model to deploy.
* Prepare data for inference. 
* Deploy the model for batch inference.
* Run a batch inference job.


### 1. Setup pre-requisites
* Install dependencies
* Connect to AzureML Workspace. Learn more at [set up SDK authentication](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-setup-authentication?tabs=sdk). Replace  `<WORKSPACE_NAME>`, `<RESOURCE_GROUP>` and `<SUBSCRIPTION_ID>` below.
* Check or create compute.
* Connect to `azureml` system registry

In [7]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential, ClientSecretCredential
from azure.ai.ml.entities import AmlCompute
import time

try:
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    credential = InteractiveBrowserCredential()

workspace_ml_client = MLClient(
        credential,
        subscription_id =  "21d8f407-c4c4-452e-87a4-e609bfb86248", #"<SUBSCRIPTION_ID>"
        resource_group_name =  "rg-contoso-819prod", #"<RESOURCE_GROUP>",
        workspace_name =  "mlw-contoso-819prod", #"WORKSPACE_NAME>",
    )
# the models, fine tuning pipelines and environments are available in the AzureML system registry, "azureml-preview"
registry_ml_client = MLClient(credential, registry_name="azureml-preview")

compute_cluster = "gpu-cluster-big"
try:
    compute = workspace_ml_client.compute.get(compute_cluster)
except Exception as ex:
    compute = AmlCompute(
        name=compute_cluster,
        size="Standard_ND40rs_v2",
        max_instances=2,  # For multi node training set this to an integer value more than 1
    )
    workspace_ml_client.compute.begin_create_or_update(compute).wait()

# genrating a unique timestamp that can be used for names and versions that need to be unique
timestamp = str(int(time.time())) 


### 2. Pick a model to deploy

Browse models in the Model Catalog in the AzureML Studio, filtering by the `text-classification` task. In this example, we use the `microsoft-deberta-base-mnli` model. If you have opened this notebook for a different model, replace the model name and version accordingly. 

In [2]:
model_name = "microsoft-deberta-base-mnli"
model_version = "2"
foundation_model=registry_ml_client.models.get(model_name, model_version)
print ("\n\nUsing model name: {0}, version: {1}, id: {2} for fine tuning".format(foundation_model.name, foundation_model.version, foundation_model.id))



Using model name: microsoft-deberta-base-mnli, version: 2, id: azureml://registries/azureml-preview/models/microsoft-deberta-base-mnli/versions/2 for fine tuning


### 3. Prepare data for inference.

A copy of the MNLI is available in the [ glue-mnli](./glue-mnli/) folder. The next few cells show basic data preparation:
* Visualize some data rows
* Replace numerical categories in data with the actual string labels. This mapping is available in the [./glue-mnli/label.json](./glue-mnli/label.json). This step is needed because the selected models will return labels such `CONTRADICTION`, `CONTRADICTION`, etc. when running prediction. If the labels in your ground truth data are left as `0`, `1`, `2`, etc., then they would not match with prediction labels returned by the models.
* The dataset contains `premise` and `hypothesis` as two different columns. However, the models expect a single string for prediction in the format `[CLS] <premise text> [SEP] <hypothesis text> [SEP]`. Hence we merge the columns and drop the original columns.
* We want this sample to run quickly, so save smaller dataset containing 10% of the original. 

In [ ]:
import os
dataset_dir = "./glue-mnli-dataset"
data_file="train.jsonl"

# load the train.jsonl file into a pandas dataframe and show the first 5 rows
import pandas as pd
pd.set_option('display.max_colwidth', 0) # set the max column width to 0 to display the full text
df = pd.read_json(os.path.join(dataset_dir,data_file), lines=True)
df.head()

# load the id2label json element of the label.json file into pandas table with keys as 'label' column of int64 type and values as 'label_string' column as string type
import json
label_file="label.json"
with open(os.path.join(dataset_dir,label_file)) as f:
    id2label = json.load(f)
    id2label = id2label['id2label']
    label_df = pd.DataFrame.from_dict(id2label, orient='index', columns=['label_string'])
    label_df['label'] = label_df.index.astype('int64')
    label_df = label_df[['label', 'label_string']]

# join the train, validation and test dataframes with the id2label dataframe to get the label_string column
df =df.merge(label_df, on='label', how='left')
# concat the premise and hypothesis columns to with "[CLS]" in the beginning and "[SEP]" in the middle and end to get the text column
df['text'] = "[CLS] " + df['premise'] + " [SEP] " + df['hypothesis'] + " [SEP]"
# drop the idx, premise and hypothesis columns as they are not needed
df = df.drop(columns=['idx', 'premise', 'hypothesis', 'label'])
# rename the label_string column to ground_truth_label
df = df.rename(columns={'label_string': 'ground_truth_label'})
# keep only the text column in batch_df dataframe
batch_df = df[['text']]
# rename text column to input_string
batch_df = batch_df.rename(columns={'text': 'input_string'})
# save 10 of the rows in batch_df dataframe to a csv file named small_batch_train.csv in the ./dataset_dir folder
batch_df.sample(frac=0.1).to_csv(os.path.join(dataset_dir,"small_batch_train.csv"), index=False)



## 3. Create Batch Endpoint

Batch endpoints are endpoints that are used batch inferencing on large volumes of data over a period of time. Batch endpoints receive pointers to data and run jobs asynchronously to process the data in parallel on compute clusters. Batch endpoints store outputs to a data store for further analysis.

To create an online endpoint we will use `BatchEndpoint`. This class allows user to configure the following key aspects:
- `name` - Name of the endpoint. Needs to be unique at the Azure region level
- `auth_mode` - The authentication method for the endpoint. Currently only Azure Active Directory (Azure AD) token-based (`aad_token`) authentication is supported. 
- `defaults` - Default settings for the endpoint.
   - `deployment_name` - Name of the deployment that will serve as the default deployment for the endpoint.
- `description`- Description of the endpoint.



In [4]:
from azure.ai.ml.entities import BatchEndpoint, BatchDeployment, Model, AmlCompute, Data
batch_endpoint_name = "entail-contra-" + timestamp
endpoint = BatchEndpoint(
    name=batch_endpoint_name,
    description="Batch endpoint for " + foundation_model.name + ", to detect entailment v/s contradiction",
)

endpoint=workspace_ml_client.batch_endpoints.begin_create_or_update(endpoint).result()

## 4. Create a batch deployment

A deployment is a set of resources required for hosting the model that does the actual inferencing. We will create a deployment for our endpoint using the `BatchDeployment` class. MLflow models don't require an scoring script.

In [10]:
from azure.ai.ml.constants import BatchDeploymentOutputAction
from azure.ai.ml.entities import BatchRetrySettings

deployment = BatchDeployment(
    name="demo",
    description="Batch deployment for " + foundation_model.name + ", to detect entailment v/s contradiction",
    endpoint_name=endpoint.name,
    model=foundation_model,
    compute=compute_cluster,
    instance_count=2,
    max_concurrency_per_instance=2,
    mini_batch_size=10,
    output_action=BatchDeploymentOutputAction.APPEND_ROW,
    output_file_name="predictions.csv",
    retry_settings=BatchRetrySettings(max_retries=3, timeout=300),
    logging_level="info",
)
deployment=workspace_ml_client.batch_deployments.begin_create_or_update(deployment).result()

In [20]:
# set demo deployment as default
endpoint = workspace_ml_client.batch_endpoints.get(batch_endpoint_name)
endpoint.defaults.deployment_name = "demo"
workspace_ml_client.batch_endpoints.begin_create_or_update(endpoint).result()

endpoint = workspace_ml_client.batch_endpoints.get(batch_endpoint_name)
print(f"The default deployment is {endpoint.defaults.deployment_name}")

Let's get a reference of the new data asset:

In [27]:
from azure.ai.ml import Input
input = Input(type=AssetTypes.URI_FOLDER, path=os.path.join(dataset_dir,"small_batch_train.csv"))

#### 4.6.3 Invoke the deployment

Using the `MLClient` created earlier, we will get a handle to the endpoint. The endpoint can be invoked using the `invoke` command with the following parameters:
- `name` - Name of the endpoint
- `input_path` - Path where input data is present

In [28]:
job = workspace_ml_client.batch_endpoints.invoke(endpoint_name=endpoint.name, input=input)

Uploading small_batch_train.csv (< 1 MB): 100%|██████████| 739k/739k [00:00<00:00, 1.26MB/s]




Notice how we are not indicating the deployment name in the invoke operation. That's because the endpoint automatically routes the job to the default deployment. Since our endpoint only has one deployment, then that one is the default one. You can target an specific deployment by indicating the argument/parameter `deployment_name`.

#### 4.6.4 Get the details of the invoked job

Let us get details and logs of the invoked job:

In [31]:
workspace_ml_client.jobs.stream(job.name)

RunId: batchjob-66d5c66f-bcf3-44c1-b5b9-0954fe6cd2a1
Web View: https://ml.azure.com/runs/batchjob-66d5c66f-bcf3-44c1-b5b9-0954fe6cd2a1?wsid=/subscriptions/21d8f407-c4c4-452e-87a4-e609bfb86248/resourcegroups/rg-contoso-819prod/workspaces/mlw-contoso-819prod

Streaming logs/azureml/executionlogs.txt

[2023-04-07 20:08:55Z] Submitting 1 runs, first five are: 43c597fb:cc7e7651-6e92-4d56-8bd0-7b81e7ef9a61
[2023-04-07 20:22:50Z] Execution of experiment failed, update experiment status and cancel running nodes.

Execution Summary
RunId: batchjob-66d5c66f-bcf3-44c1-b5b9-0954fe6cd2a1
Web View: https://ml.azure.com/runs/batchjob-66d5c66f-bcf3-44c1-b5b9-0954fe6cd2a1?wsid=/subscriptions/21d8f407-c4c4-452e-87a4-e609bfb86248/resourcegroups/rg-contoso-819prod/workspaces/mlw-contoso-819prod


JobException: Exception : 
 {
    "error": {
        "code": "UserError",
        "message": "Pipeline has failed child jobs. For more details and logs, please go to the job detail page and check the child jobs.",
        "message_format": "Pipeline has failed child jobs. {0}",
        "message_parameters": {},
        "reference_code": "PipelineHasStepJobFailed",
        "details": []
    },
    "environment": "eastus",
    "location": "eastus",
    "time": "2023-04-07T20:22:50.613629Z",
    "component_name": ""
} 

We can wait for the job to finish using the following code:

### 4.7 Exploring the results

#### 4.7.1 Download the results

The deployment creates a child job that executes the scoring. We can get the details of it using the following code:

In [ ]:
scoring_job = list(ml_client.jobs.list(parent_job_name=job.name))[0]

The outputs generated by the deployment job will be placed in an output named `score`:

In [ ]:
ml_client.jobs.download(name=scoring_job.name, download_path=".", output_name="score")

We can read this data using pandas library:

> Tip: Notice how the CSV is read as opposite to use directly `pd.read_csv`.

In [ ]:
with open("named-outputs/score/predictions.csv", "r") as f:
    data = f.read()

In [ ]:
from ast import literal_eval
import pandas as pd

score = pd.DataFrame(
    literal_eval(data.replace("\n", ",")), columns=["file", "prediction"]
)
score

## 5. Clean up resources

Clean-up the resources created. 

In [ ]:
ml_client.batch_endpoints.begin_delete(endpoint_name)